In [18]:
################### 데이터 불러오기
import numpy
import pandas as pd
import pygad

"""
The y=target is to maximize this equation ASAP:
    y = w1x1+w2x2+w3x3+w4x4+w5x5+6wx6
    where (x1,x2,x3,x4,x5,x6)=(4,-2,3.5,5,-11,-4.7)
    What are the best values for the 6 weights w1 to w6?
    We are going to use the genetic algorithm for the best possible values after a number of generations.
"""
# data
restaurants = pd.read_csv("https://raw.githubusercontent.com/xoyeon/PDPTW/main/grubhub/0o100t100s1p100/restaurants.txt", sep = '\t')
orders = pd.read_csv("https://raw.githubusercontent.com/xoyeon/PDPTW/main/grubhub/0o100t100s1p100/orders.txt", sep = '\t')
couriers = pd.read_csv("https://raw.githubusercontent.com/xoyeon/PDPTW/main/grubhub/0o100t100s1p100/couriers.txt", sep = '\t')
rest_orders = pd.merge(restaurants, orders, how='inner', on=['restaurant'])
rest_orders.columns = ['restaurant', 'rx', 'ry', 'order', 'ox', 'oy', 'placement_time', 'ready_time']

# 주문 건
# 점심시간 >=90, <300 / 주문 건 수 136 명, 근무중인 라이더 24 명
# 저녁시간 >=450, <700 / 주문 건 수 312 명 근무중인 라이더 53 명
order = rest_orders[(rest_orders['placement_time']>=450) & (rest_orders['ready_time']<700)]
print("주문 건 수", len(order), "명")
# 근무중인 라이더
rider = couriers[(couriers['on_time']>=450) & (couriers['off_time']<700)]
data_len = len(rider)
print("근무중인 라이더", len(rider), "명")

주문 건 수 296 명
근무중인 라이더 52 명


In [20]:
order = rest_orders[(rest_orders['placement_time']>=450) & (rest_orders['ready_time']<700)].index
rider = couriers[(couriers['on_time']>=450) & (couriers['off_time']<700)].index
print(order)
print(rider)

Int64Index([  1,   3,   4,   5,   6,   8,  12,  13,  14,  15,
            ...
            486, 487, 488, 490, 491, 492, 493, 494, 495, 502],
           dtype='int64', length=296)
Int64Index([40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56,
            57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73,
            74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90,
            91],
           dtype='int64')


In [23]:
non_peak = rest_orders.drop(order)
non_peak.to_csv("non_rest.csv")

In [25]:
non_rider = couriers.drop(rider)
non_rider.to_csv("non_rider.csv")

In [26]:
non_rider

,courier,x,y,on_time,off_time
0,c1,11491,2806,0,90
1,c2,7950,223,0,90
2,c3,2818,10568,30,210
3,c4,12413,7222,90,270
4,c5,3572,3264,90,270
...,...,...,...,...,...
108,c109,10855,7784,690,810
109,c110,7038,5694,690,840
110,c111,5013,2778,690,810
111,c112,8236,1518,690,810


In [28]:
non_peak

,restaurant,rx,ry,order,ox,oy,placement_time,ready_time
0,r1,8708,5633,o1,9131,7497,743,753
2,r1,8708,5633,o21,7377,5287,336,346
7,r1,8708,5633,o54,8230,8128,740,750
9,r1,8708,5633,o69,9990,6549,680,700
10,r1,8708,5633,o81,9098,8163,776,786
...,...,...,...,...,...,...,...,...
499,r111,9738,7858,o419,9778,7773,96,106
500,r112,8884,7405,o426,10229,7353,124,154
501,r113,796,6628,o430,953,4911,73,104
503,r115,9843,9303,o451,9671,8669,741,771


In [ ]:
delivered_num = numpy.random.uniform(low=3, high=25, size=data_len).astype(int)
dist_x = numpy.array(rider['x'])
dist_y = numpy.array(rider['y'])
on_time = numpy.array(rider['on_time'])
off_time = numpy.array(rider['off_time'])
speed = numpy.random.uniform(low=30, high=60, size=data_len).astype(int)
trustiness = numpy.full(data_len, 0.90)

new_population = numpy.stack((delivered_num, dist_x, dist_y, on_time, off_time, speed, trustiness), axis=0)

In [ ]:
# Function inputs.
function_inputs = new_population

# desired_output = 44 # Function output.
desired_output = numpy.array([[0],[2875],[3459],[400],[700],[1],[1]])

# delivered_num, dist_x, dist_y, on_time, off_time, speed, trustiness

def fitness_func(solution, solution_idx):
    solution = solution.reshape(7,52)
    
    std = numpy.std(solution[0])
    log_std = numpy.log1p(std)
    fit_std = numpy.abs(log_std - desired_output[0])

    speed = solution[5]
    sc_speed = ((speed - min(speed)) / (max(speed) - min(speed)))
    fit_speed = numpy.abs(sc_speed - desired_output[5])

    trust = (solution[4]- solution[3]) / solution[0]
    sc_trust = ((trust - min(trust)) / (max(trust) - min(trust)))
    fit_trust = numpy.abs(sc_trust - desired_output[6])

    fitness = 0.5*(1/fit_std) + 0.5*(fit_speed + fit_trust)
    return fitness[0]

##################
fitness_function = fitness_func

def on_start(ga_instance):
    print("on_start()")

def on_fitness(ga_instance, population_fitness):
    print("on_fitness()")

def on_parents(ga_instance, selected_parents):
    print("on_parents()")

def on_crossover(ga_instance, offspring_crossover):
    print("on_crossover()")

def on_mutation(ga_instance, offspring_mutation):
    print("on_mutation()")

def on_generation(ga_instance):
    print("on_generation()")

def on_stop(ga_instance, last_population_fitness):
    print("on_stop()")

########### pop 100
ga_instance = pygad.GA(num_generations=1000,
                       num_parents_mating=4,
                       fitness_func=fitness_function,
                       sol_per_pop=500,
                       num_genes=len(function_inputs)*len(function_inputs[0]),
                       save_best_solutions=True,
                       on_start=on_start,
                       on_fitness=on_fitness,
                       on_parents=on_parents,
                       on_crossover=on_crossover,
                       on_mutation=on_mutation,
                       on_generation=on_generation,
                       on_stop=on_stop)

ga_instance.run()

# After the generations complete, some plots are showed that summarize the how the outputs/fitenss values evolve over generations.

ga_instance.plot_fitness()

###################

# Returning the details of the best solution.
solution, solution_fitness, solution_idx = ga_instance.best_solution()
solution=solution.reshape(7,52)
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))
print("Index of the best solution : {solution_idx}".format(solution_idx=solution_idx))

prediction = numpy.sum(numpy.array(function_inputs)*solution.reshape(7,52))
print("Predicted output based on the best solution : {prediction}".format(prediction=prediction))

if ga_instance.best_solution_generation != 1:
    print("Best fitness value reached after {best_solution_generation} generations.".format(best_solution_generation=ga_instance.best_solution_generation))

print("근무시간 내에 배달하는 건수의 표준편차", numpy.std(solution[0] / (solution[4] - solution[3])))
print("신뢰도", numpy.sum(solution[6]))